In [17]:
import json

import pandas as pd
import numpy as np
import datetime as dt
import plotly.graph_objects as go

In [23]:
with open("../results/BTCUSDT_candlestick.json",'r') as f:
    data = json.loads(f.read())
data = pd.json_normalize(data,record_path=['BTCUSDT'])

In [25]:
float_variables = ["OpenPrice","HighPrice","LowPrice","ClosePrice","Volume","QuoteAssetVolume"]
datetime_variables = ["openTime","closeTime"]

for i in range(len(float_variables)):
    data[float_variables[i]] = data[float_variables[i]].astype(float)

for i in range(len(datetime_variables)):
    data[datetime_variables[i]] = pd.to_datetime(data[datetime_variables[i]])

In [22]:
data.head()

,openTime,OpenPrice,HighPrice,LowPrice,ClosePrice,Volume,closeTime,QuoteAssetVolume,nTrades
0,2023-10-01 14:00:00,27192.04,27229.29,27083.24,27119.79,996.38542,2023-10-01 14:59:59,2.705339e+07,43790
1,2023-10-01 15:00:00,27119.79,27155.85,27107.01,27153.18,541.59390,2023-10-01 15:59:59,1.469446e+07,29637
2,2023-10-01 16:00:00,27153.17,27155.00,27082.38,27112.19,585.50462,2023-10-01 16:59:59,1.588032e+07,29652
3,2023-10-01 17:00:00,27112.20,27140.45,27035.78,27072.35,730.04943,2023-10-01 17:59:59,1.977801e+07,34610
4,2023-10-01 18:00:00,27072.34,27100.01,27065.07,27096.13,438.07759,2023-10-01 18:59:59,1.186694e+07,25812


In [26]:
fig = go.Figure(
    data=[go.Candlestick(x=data["openTime"],
                         open=data["OpenPrice"], high=data["HighPrice"],
                         low=data["LowPrice"],close=data["ClosePrice"])
                         ])

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# *STRATEGY*

In [64]:
data = pd.read_csv("../results/strategy_1.csv",index_col='timestamp')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 2023-11-01 10:44:34 to 2023-11-01 10:54:34
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   600 non-null    float64
 1   MA_5    596 non-null    float64
 2   MA_25   576 non-null    float64
 3   MA_50   551 non-null    float64
 4   signal  576 non-null    object 
dtypes: float64(4), object(1)
memory usage: 28.1+ KB


In [65]:
results = []

wallet = 100
prev_signal = None

for row in range(len(data)):
    
    if data.iloc[row, 4] == "buy" and prev_signal != "buy":
        wallet = np.divide(wallet,data.iloc[row,0])
        prev_signal = data.iloc[row, 4]

        results.append({"index": data.index[row], "wallet": wallet * data.iloc[row,0]})

    elif data.iloc[row, 4] == "sell" and prev_signal not in ["sell",None]:
        wallet = np.multiply(wallet,data.iloc[row,0])
        prev_signal = data.iloc[row, 4]

        results.append({"index": data.index[row], "wallet": wallet})

    else:
        continue

df = pd.DataFrame(results)

with open("../results/strategy_1_results.csv", 'w') as csv_file:
    df.to_csv(csv_file, index=False, header=True)


In [67]:
data = pd.read_csv("../results/strategy_1_results.csv", index_col='index')
data

,wallet
index,
2023-11-01 10:46:10,100.000000
2023-11-01 10:46:16,99.984384
2023-11-01 10:46:52,99.984384
2023-11-01 10:47:20,99.984384
2023-11-01 10:47:21,99.984384
2023-11-01 10:47:26,99.984355
2023-11-01 10:47:28,99.984355
2023-11-01 10:47:29,99.984326
2023-11-01 10:47:30,99.984326


In [76]:
t0 = data.iloc[0,0]
t1 = data.iloc[-1,0]

print(t0)
print(t1)


100.0
100.01137697219264


In [81]:
rentabilidad = (t1 - t0)/t0
format(rentabilidad, ".2%")

'0.01%'